# Section 6: Estimator Primitive (12% of Exam)

> **Exam Weight**: ~8 questions | **Difficulty**: Medium-High | **Must Master**: ✅✅✅

---

## Part 1: What is Estimator?

**Estimator** calculates expectation values ⟨ψ|O|ψ⟩ - the average value of an observable on a quantum state.

```
Formula: ⟨O⟩ = ⟨ψ|O|ψ⟩

Example: Z operator on |+⟩
     ┌───┐
q: ──┤ H ├──   Observable: Z
     └───┘
     
|+⟩ = (|0⟩ + |1⟩)/√2
⟨Z⟩ = (+1)·(1/2) + (-1)·(1/2) = 0
```

**Two Types**:
1. **StatevectorEstimator** - Local simulation (exact)
2. **Estimator** (Runtime) - Real IBM hardware

**CRITICAL**: Circuit does NOT need measurements!

### ⚠️ EXAM TRAP: Estimator vs Sampler

| Feature | Sampler | Estimator |
|---------|---------|------------|
| Purpose | Measurement counts | Expectation values |
| Needs measurements? | ✅ YES | ❌ NO |
| Output | `{'00': 512}` | `⟨O⟩ = 0.73` |
| Output type | dict | float |
| Access | `.data.meas.get_counts()` | `.data.evs` |
| Use case | Get bitstrings | Calculate energy |

**Memory Aid**: "Sampler = Sample bitstrings, Estimator = Estimate ⟨O⟩"

## Part 2: Local Estimator - Basic Usage

In [ ]:
# Basic Estimator Pattern

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# Create circuit (NO measurements!)
qc = QuantumCircuit(1)
qc.h(0)  # |+⟩ state

print("Circuit:")
print(qc.draw())

# Define observable
observable = SparsePauliOp('Z')  # Pauli Z

# Create Estimator
estimator = StatevectorEstimator()

# Run circuit with observable
job = estimator.run([(qc, observable)])

# Get expectation value
result = job.result()
expectation = result[0].data.evs


print(f"\n⟨Z⟩ = {expectation}")
print("\n✓ |+⟩ state: equal superposition of |0⟩ and |1⟩")
print("✓ ⟨Z⟩ = (+1)(0.5) + (-1)(0.5) = 0")

### 🎯 EXAM CRITICAL: Result Access Pattern

**MEMORIZE THIS**:
```python
expectation = result[0].data.evs
              ↑      ↑    ↑
              │      │    └─ Expectation values (plural!)
              │      └───── Data attribute
              └─────────── Circuit index
```

**Note**: `.evs` (plural) NOT `.ev`!

## Part 3: SparsePauliOp - Observable Construction

In [ ]:
# SparsePauliOp Construction Examples

from qiskit.quantum_info import SparsePauliOp

# Single Pauli operators
Z = SparsePauliOp('Z')
X = SparsePauliOp('X')
Y = SparsePauliOp('Y')

print("Single Pauli operators:")
print(f"Z = {Z}")
print(f"X = {X}")

# Multi-qubit operators (RIGHT TO LEFT!)
ZZ = SparsePauliOp('ZZ')  # Z⊗Z
XX = SparsePauliOp('XX')  # X⊗X
ZI = SparsePauliOp('ZI')  # Z on q0, I on q1
IZ = SparsePauliOp('IZ')  # I on q0, Z on q1

print("\nMulti-qubit operators:")
print(f"ZZ = {ZZ}")
print(f"ZI = {ZI}")

# With coefficients - Hamiltonian construction

H = SparsePauliOp(['Z', 'X'], coeffs=[0.5, 0.3])
print(f"\nHamiltonian H = 0.5*Z + 0.3*X:")
print(f"H = {H}")

# Alternative: from_list
H2 = SparsePauliOp.from_list([('Z', 0.5), ('X', 0.3)])
print(f"\nSame using from_list: {H2}")

# Operator arithmetic also works
H3 = 0.5 * SparsePauliOp('Z') + 0.3 * SparsePauliOp('X')
print(f"Same using arithmetic: {H3}")

### ⚠️ EXAM TRAP: Qubit Ordering

**CRITICAL**: Pauli strings are **RIGHT TO LEFT** (tensor product order)!

```python
# 'ZX' means Z⊗X
SparsePauliOp('ZX')
#  ┌─┬─┐
#  │Z│X│  = Z on qubit 0, X on qubit 1
#  └─┴─┘
#  q0 q1

# 'XYZ' means X⊗Y⊗Z
SparsePauliOp('XYZ')
# Qubit 0: X
# Qubit 1: Y
# Qubit 2: Z
```

**Memory Aid**: "Pauli string = tensor product, first char = first qubit"

## ⚠️ EXAM CRITICAL: Estimator PUB Format

**PUB = (circuit, observable, parameter_values, precision)**

| Component | Required | Type | Description |
|-----------|----------|------|-------------|
| `circuit` | ✅ YES | QuantumCircuit | NO measurements! |
| `observable` | ✅ YES | SparsePauliOp | What to measure |
| `parameter_values` | Optional | list | Values for Parameters |
| `precision` | Optional | float | Target precision |

### PUB Format Table

| Scenario | Format | Example |
|----------|--------|---------|
| Basic | `(circuit, observable)` | `estimator.run([(qc, obs)])` |
| With params | `(circuit, observable, params)` | `estimator.run([(qc, obs, [0.5])])` |
| With precision | `(circuit, observable, None, prec)` | `estimator.run([(qc, obs, None, 0.01)])` |
| All options | `(circuit, observable, params, prec)` | `estimator.run([(qc, obs, [0.5], 0.01)])` |

In [ ]:
# Estimator PUB Format Examples

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# Create circuits
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

theta = Parameter('θ')
qc_param = QuantumCircuit(1)
qc_param.ry(theta, 0)

obs = SparsePauliOp('ZZ')
obs_single = SparsePauliOp('Z')

estimator = StatevectorEstimator()

print("="*60)
print("ESTIMATOR PUB FORMATS")
print("PUB = (circuit, observable, parameter_values, precision)")
print("="*60)

# Example 1: Basic - (circuit, observable)
print("\n[1] Basic: (circuit, observable)")
pub1 = (qc, obs)
job = estimator.run([pub1])
print(f"    estimator.run([{pub1}])")
print(f"    Result: ⟨ZZ⟩ = {job.result()[0].data.evs:.4f}")

# Example 2: With parameters - (circuit, observable, params)
print("\n[2] With params: (circuit, observable, [param_values])")
pub2 = (qc_param, obs_single, [0.5])
job = estimator.run([pub2])
print(f"    estimator.run([(qc_param, obs, [0.5])])")
print(f"    Result: ⟨Z⟩ = {job.result()[0].data.evs:.4f}")

# Example 3: Multiple parameter sets (batch)
print("\n[3] Batch: Multiple parameter values")
theta_values = [[0.0], [1.57], [3.14]]  # 0, π/2, π
pubs = [(qc_param, obs_single, val) for val in theta_values]
job = estimator.run(pubs)
for i, val in enumerate(theta_values):
    ev = job.result()[i].data.evs
    print(f"    θ = {val[0]:.2f}: ⟨Z⟩ = {ev:.4f}")

# Example 4: Multiple observables for same circuit
print("\n[4] Multiple observables: Same circuit, different obs")
observables = [SparsePauliOp('ZZ'), SparsePauliOp('XX'), SparsePauliOp('YY')]
pubs = [(qc, obs) for obs in observables]
job = estimator.run(pubs)
for i, obs_label in enumerate(['ZZ', 'XX', 'YY']):
    ev = job.result()[i].data.evs
    print(f"    ⟨{obs_label}⟩ = {ev:.4f}")

print("\n" + "="*60)
print("COMMON MISTAKES:")
print("❌ estimator.run([circuit])  # Missing observable!")
print("❌ estimator.run([(circuit, 'ZZ')])  # String not SparsePauliOp!")
print("✅ estimator.run([(circuit, SparsePauliOp('ZZ'))])")
print("="*60)

## Part 4: Multi-Qubit Observables

In [ ]:
# Multi-Qubit Observables and Hamiltonians

from qiskit.quantum_info import SparsePauliOp

# Two-qubit Pauli operators
ZZ = SparsePauliOp('ZZ')  # Z⊗Z
XX = SparsePauliOp('XX')  # X⊗X
YY = SparsePauliOp('YY')  # Y⊗Y

print("Two-qubit operators:")
print(f"ZZ = {ZZ}")
print(f"XX = {XX}")

# Mixed operators
ZX = SparsePauliOp('ZX')  # Z on q0, X on q1
XZ = SparsePauliOp('XZ')  # X on q0, Z on q1

print(f"\nMixed: ZX = {ZX}")
print("Note: 'ZX' = Z⊗X, first char (Z) on qubit 0")
# Hamiltonian: H = 0.5*ZZ + 0.3*XX
H = SparsePauliOp(['ZZ', 'XX'], coeffs=[0.5, 0.3])
print(f"\nHamiltonian H = 0.5*ZZ + 0.3*XX:")
print(H)

# H2 Molecule Hamiltonian (common exam example!)
H2_molecule = SparsePauliOp(
    ["II", "ZI", "IZ", "ZZ", "XX"],
    [-1.05, 0.39, 0.39, -0.01, 0.18]
)
print(f"\nH2 Molecule Hamiltonian (5 terms):")
print(H2_molecule)

## Part 5: Estimator with Pauli Operators

In [ ]:
# Estimator with Multiple Observables - Bell State

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# Create Bell state |Φ+⟩ = (|00⟩ + |11⟩)/√2
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

print("Bell State Circuit:")
print(qc.draw())

# Different observables to measure
observables = [
    SparsePauliOp('ZZ'),
    SparsePauliOp('XX'),
    SparsePauliOp('YY'),
    SparsePauliOp('ZI'),
]

estimator = StatevectorEstimator()

# Run all observables at once
pubs = [(qc, obs) for obs in observables]
job = estimator.run(pubs)
result = job.result()

# Extract expectation values
print("\nBell State Expectation Values:")
print(f"⟨ZZ⟩ = {result[0].data.evs:.3f}")  # 1.0 (correlated)
print(f"⟨XX⟩ = {result[1].data.evs:.3f}")  # 1.0 (correlated)
print(f"⟨YY⟩ = {result[2].data.evs:.3f}")  # -1.0 (anti-correlated)
print(f"⟨ZI⟩ = {result[3].data.evs:.3f}")  # 0.0 (maximally mixed)

### Bell State Expectation Values (MEMORIZE!)

```
Bell |Φ+⟩ = (|00⟩ + |11⟩)/√2

⟨ZZ⟩ = 1   ✓ Correlated in Z
⟨XX⟩ = 1   ✓ Correlated in X
⟨YY⟩ = -1  ✓ Anti-correlated in Y
⟨ZI⟩ = 0   ✓ Single qubit: maximally mixed
⟨IZ⟩ = 0   ✓ Single qubit: maximally mixed
```

## Part 6: Hamiltonian Expectation Values

In [ ]:
# Hamiltonian Expectation Values

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# Single-qubit Hamiltonian: H = Z + X
H1 = SparsePauliOp.from_list([('Z', 1.0), ('X', 1.0)])

# Test with |+⟩ state
qc1 = QuantumCircuit(1)
qc1.h(0)

estimator = StatevectorEstimator()
job = estimator.run([(qc1, H1)])
energy1 = job.result()[0].data.evs

print("Single-qubit: H = Z + X on |+⟩")
print(f"⟨H⟩ = {energy1:.4f}")
print("Breakdown: ⟨Z⟩ = 0, ⟨X⟩ = 1, so ⟨H⟩ = 0 + 1 = 1")

# Two-qubit Hamiltonian: H = ZZ + XX
H2 = SparsePauliOp.from_list([('ZZ', 1.0), ('XX', 1.0)])

# Test with Bell state
qc2 = QuantumCircuit(2)
qc2.h(0)
qc2.cx(0, 1)

job = estimator.run([(qc2, H2)])
energy2 = job.result()[0].data.evs

print(f"\nTwo-qubit: H = ZZ + XX on Bell state")
print(f"⟨H⟩ = {energy2:.4f}")
print("Breakdown: ⟨ZZ⟩ = 1, ⟨XX⟩ = 1, so ⟨H⟩ = 1 + 1 = 2")

### Hamiltonian Construction Examples

```python
# Ising model: H = -J*ZZ - h*Z
J, h = 1.0, 0.5
H = SparsePauliOp(['ZZ', 'ZI'], coeffs=[-J, -h])

# Heisenberg: H = XX + YY + ZZ
H = SparsePauliOp(['XX', 'YY', 'ZZ'], coeffs=[1, 1, 1])

# Transverse field Ising: H = -ZZ - hX
H = SparsePauliOp(['ZZ', 'XI'], coeffs=[-1, -h])
```

## Part 7: Parameterized Circuits with Estimator

In [ ]:
# Parameterized Circuits with Estimator

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
import numpy as np

# Create parameterized circuit
theta = Parameter('θ')
qc = QuantumCircuit(1)
qc.ry(theta, 0)

# Observable
Z_obs = SparsePauliOp('Z')

estimator = StatevectorEstimator()

# Measure ⟨Z⟩ at different angles
angles = [0, np.pi/4, np.pi/2, np.pi]

print("Parameterized Circuit Results:")
print("θ (rad)    ⟨Z⟩      Expected (cos θ)")
print("-" * 40)

for angle in angles:
    qc_bound = qc.assign_parameters({theta:angle})
    job = estimator.run([(qc_bound, Z_obs)])
    result = job.result()
    ev = result[0].data.evs
    expected = np.cos(angle)
    print(f'{angle:6.2f}    {ev:7.4f}    {expected:7.4f}')

print("\n✓ Bind parameters with assign_parameters()")
print("✓ Same pattern as Sampler")
print("✓ Critical for VQE optimization")

In [ ]:
# Runtime Estimator Pattern (Real Hardware)
# This is reference code - requires IBM Quantum account

print("""
Runtime Estimator Pattern (for real hardware):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator, Session
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Setup service
service = QiskitRuntimeService()
backend = service.least_busy(simulator=False, operational=True)

# Create circuit (NO measurements for Estimator)
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Define observable
observable = SparsePauliOp('ZZ')

# Transpile for target backend
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
qc_transpiled = pm.run(qc)
observable_mapped = observable.apply_layout(qc_transpiled.layout)

# Run with Session
with Session(backend=backend) as session:
    estimator = Estimator(mode=session)
    job = estimator.run([(qc_transpiled, observable_mapped)])
    result = job.result()
    expectation = result[0].data.evs
""")

print("\n🎯 Key Differences from Local:")
print("  ✓ Import from qiskit_ibm_runtime")
print("  ✓ Need QiskitRuntimeService and backend")
print("  ✓ Transpile circuit before running")
print("  ✓ Apply layout to observable")
print("  ✓ Use Session for efficiency")
print("  ✓ Same result access: result[0].data.evs")

## Part 9: Common Patterns

In [ ]:
# Common Estimator Patterns - All Executable

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
import numpy as np

estimator = StatevectorEstimator()

# Create test circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Pattern 1: Single observable
print("Pattern 1: Single observable")
obs = SparsePauliOp('ZZ')
job = estimator.run([(qc, obs)])
ev = job.result()[0].data.evs
print(f"⟨ZZ⟩ = {ev:.4f}\n")

# Pattern 2: Multiple observables (batch)
print("Pattern 2: Multiple observables")
obs_list = [SparsePauliOp('ZZ'), SparsePauliOp('XX'), SparsePauliOp('YY')]
job = estimator.run([(qc, obs) for obs in obs_list])
result = job.result()
for i, name in enumerate(['ZZ', 'XX', 'YY']):
    print(f"⟨{name}⟩ = {result[i].data.evs:.4f}")

# Pattern 3: Hamiltonian (VQE-style)
print("\nPattern 3: Hamiltonian")
H = SparsePauliOp(['ZZ', 'XX'], coeffs=[1.0, 0.5])
job = estimator.run([(qc, H)])
energy = job.result()[0].data.evs
print(f"⟨H⟩ = ⟨1.0*ZZ + 0.5*XX⟩ = {energy:.4f}")

# Pattern 4: Parameter sweep
print("\nPattern 4: Parameter sweep")
theta = Parameter('θ')
qc_param = QuantumCircuit(1)
qc_param.ry(theta, 0)

angles = np.linspace(0, np.pi, 5)
circuits = [qc_param.assign_parameters({theta: a}) for a in angles]
observables = [SparsePauliOp('Z')] * len(circuits)

job = estimator.run([(c, o) for c, o in zip(circuits, observables)])
result = job.result()
print("θ         ⟨Z⟩")
for i, angle in enumerate(angles):
    print(f"{angle:.2f}      {result[i].data.evs:.4f}")

## 📝 Practice Questions

### Question 1: Result Access

**How do you extract expectation values from Estimator results?**

A) `result.data.evs`  
B) `result[0].data.evs`  
C) `result[0].data.meas.get_counts()`  
D) `result[0].expectation_value()`

<details>
<summary>Answer</summary>

**B) `result[0].data.evs`**

```python
result = job.result()
expectation = result[0].data.evs
              ↑      ↑    ↑
              │      │    └─ Expectation values (plural!)
              │      └───── Data attribute
              └─────────── Circuit index
```

**Note**: `.evs` (plural) NOT `.ev`!
</details>

---

### Question 2: Qubit Ordering

**What does `SparsePauliOp('ZX')` represent?**

A) X on qubit 0, Z on qubit 1  
B) Z on qubit 0, X on qubit 1  
C) Z and X both on qubit 0  
D) Z⊗X with visual left-to-right order

<details>
<summary>Answer</summary>

**B) Z on qubit 0, X on qubit 1**

```python
# 'ZX' means Z⊗X (tensor product)
SparsePauliOp('ZX')
#  ┌─┬─┐
#  │Z│X│  = Z on qubit 0, X on qubit 1
#  └─┴─┘
#  q0 q1

# First character = first qubit
```

**Memory Aid**: "First char = first qubit" (tensor product order)
</details>

---

### Question 3: Measurements Required?

**Does Estimator require measurements in the circuit?**

A) Yes, like Sampler  
B) No, measurements not needed  
C) Only for real hardware  
D) Depends on observable

<details>
<summary>Answer</summary>

**B) No, measurements not needed**

```python
# CORRECT - No measurements
qc = QuantumCircuit(1)
qc.h(0)
estimator.run([(qc, SparsePauliOp('Z'))])  # ✓ Works

# WRONG - Adding measurements causes error
qc.measure_all()
estimator.run([(qc, SparsePauliOp('Z'))])  # ❌ ERROR!
```

**Key**: Sampler needs measurements, Estimator does NOT!
</details>

---

### Question 4: Hamiltonian Construction

**How do you create Hamiltonian H = 0.5*ZZ + 0.3*XX?**

A) `SparsePauliOp('ZZ + XX', coeffs=[0.5, 0.3])`  
B) `SparsePauliOp(['ZZ', 'XX'], coeffs=[0.5, 0.3])`  
C) `SparsePauliOp({'ZZ': 0.5, 'XX': 0.3})`  
D) `SparsePauliOp('ZZ') + SparsePauliOp('XX')`

<details>
<summary>Answer</summary>

**B) `SparsePauliOp(['ZZ', 'XX'], coeffs=[0.5, 0.3])`**

```python
# Method 1: List of Paulis with coefficients
H = SparsePauliOp(['ZZ', 'XX'], coeffs=[0.5, 0.3])

# Method 2: from_list with tuples
H = SparsePauliOp.from_list([('ZZ', 0.5), ('XX', 0.3)])

# Method 3: Operator arithmetic
H = 0.5 * SparsePauliOp('ZZ') + 0.3 * SparsePauliOp('XX')
```

All three methods are correct, but Method 1 is most common.
</details>

---

## ✅ Key Takeaways

### Core Concepts

1. **Estimator Purpose**
   - Compute expectation values ⟨ψ|O|ψ⟩
   - Returns float (not dict like Sampler)
   - NO measurements needed in circuit

2. **Two Types**
   - StatevectorEstimator: Local simulation
   - Estimator (Runtime): Real hardware
   - Same result access: `.data.evs`

3. **SparsePauliOp**
   - Represents observables/Hamiltonians
   - String format: 'Z', 'ZZ', 'XX'
   - Qubit ordering: RIGHT TO LEFT (tensor)
   - With coefficients: `['ZZ', 'XX'], coeffs=[a, b]`

4. **Result Access**
   - `result[i].data.evs` (plural!)
   - Returns float expectation value
   - Index [i] for multiple circuits/observables

### Critical Exam Facts

- ✅ **MEMORIZE**: `result[0].data.evs` (plural!)
- ✅ Circuit does NOT need measurements
- ✅ Qubit ordering: 'ZX' = Z on q0, X on q1
- ✅ Local: `qiskit.primitives`, Runtime: `qiskit_ibm_runtime`
- ✅ Hamiltonians: H = sum of Pauli terms
- ✅ Bell state: ⟨ZZ⟩ = ⟨XX⟩ = 1, ⟨YY⟩ = -1
- ✅ VQE uses Estimator to compute energy

### Common Traps

- ❌ Adding measurements → ✅ No measure() needed!
- ❌ `.data.ev` (singular) → ✅ `.data.evs` (plural)
- ❌ 'ZX' = X on q0, Z on q1 → ✅ Z on q0, X on q1
- ❌ Using get_counts() → ✅ Use .evs for Estimator

### Mnemonic

🧠 **"Estimator Estimates, No Measure, First First!"**

- **Estimator Estimates** - Computes ⟨O⟩
- **No Measure** - No measurements in circuit
- **First First** - First char = first qubit

**Next**: VQE (Variational Quantum Eigensolver)!

## Part 7: Practical Estimator Patterns

### Pattern 1: Multi-Observable Measurement

**EXAM TIP**: Estimator can measure multiple observables simultaneously!

In [ ]:
# Multi-observable measurement
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.quantum_info import SparsePauliOp

# Bell state
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Multiple observables
observables = [
    SparsePauliOp('ZZ'),  # Correlation
    SparsePauliOp('XX'),  # X-basis correlation
    SparsePauliOp('YY'),  # Y-basis correlation
    SparsePauliOp('ZI'),  # Single qubit Z
]

estimator = Estimator()

# Measure all simultaneously
pubs = [(qc, obs) for obs in observables]
job = estimator.run(pubs)
result = job.result()

print('Bell State Expectation Values:')
print(f'⟨ZZ⟩ = {result[0].data.evs:.3f}')  # 1.0 (perfect correlation)
print(f'⟨XX⟩ = {result[1].data.evs:.3f}')  # 1.0
print(f'⟨YY⟩ = {result[2].data.evs:.3f}')  # -1.0
print(f'⟨ZI⟩ = {result[3].data.evs:.3f}')  # 0.0

print('\n✓ Multiple observables measured in ONE job')
print('✓ Result indices match observable order')
print('✓ Bell state shows perfect correlations')

### Pattern 2: H2 Molecule Energy Calculation

**EXAM CRITICAL**: Real molecular Hamiltonian example!

In [ ]:
# H2 molecule Hamiltonian
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.quantum_info import SparsePauliOp

# H2 Hamiltonian (Jordan-Wigner encoding)
H = SparsePauliOp(
    ["II", "ZI", "IZ", "ZZ", "XX"],
    [-1.05, 0.39, 0.39, -0.01, 0.18]
)

print('H2 Molecule Hamiltonian:')
print(f'H = -1.05*II + 0.39*ZI + 0.39*IZ - 0.01*ZZ + 0.18*XX')
print(f'\nNumber of terms: {len(H)}')

# Simple trial state (not optimized)
qc = QuantumCircuit(2)
qc.x(0)  # Occupy first orbital
qc.x(1)  # Occupy second orbital

# Calculate energy
estimator = Estimator()
job = estimator.run([(qc, H)])
result = job.result()
energy = result[0].data.evs

print(f'\nEnergy of |11⟩ state: {energy:.4f} Hartree')
print('(Not ground state - needs VQE optimization!)')

print('\n✓ SparsePauliOp accepts lists of Paulis and coefficients')
print('✓ Real molecular problems use 5+ term Hamiltonians')
print('✓ This pattern appears frequently on exam!')

### Pattern 3: Standard Deviation Access

**EXAM TRAP**: `.stds` (plural) for standard deviations!

In [ ]:
# Access standard deviations
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.quantum_info import SparsePauliOp

qc = QuantumCircuit(1)
qc.h(0)

observable = SparsePauliOp('X')
estimator = Estimator()

job = estimator.run([(qc, observable)])
result = job.result()

# Access both expectation value and standard deviation
expectation = result[0].data.evs
std_dev = result[0].data.stds  # Note: plural!

print(f'⟨X⟩ = {expectation:.3f} ± {std_dev:.3f}')

print('\n⚠️ MEMORIZE: .evs (plural) and .stds (plural)')
print('✓ StatevectorEstimator has zero uncertainty (exact)')
print('✓ Runtime Estimator has measurement shot noise')

### Pattern 4: GHZ State Verification

**Exam Practice Problem**: Verify 3-qubit entanglement with Estimator

In [ ]:
# GHZ state verification
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.quantum_info import SparsePauliOp

# Create GHZ state: (|000⟩ + |111⟩)/√2
qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)

# Witnesses for genuine tripartite entanglement
witnesses = [
    SparsePauliOp('XXX'),  # All X correlations
    SparsePauliOp('ZZI'),  # Pairwise Z
    SparsePauliOp('ZIZ'),
    SparsePauliOp('IZZ'),
]

estimator = Estimator()
pubs = [(qc, w) for w in witnesses]
job = estimator.run(pubs)
result = job.result()

print('GHZ State Witnesses:')
print(f'⟨XXX⟩ = {result[0].data.evs:.3f}')  # -1.0
print(f'⟨ZZI⟩ = {result[1].data.evs:.3f}')  # 1.0
print(f'⟨ZIZ⟩ = {result[2].data.evs:.3f}')  # 1.0
print(f'⟨IZZ⟩ = {result[3].data.evs:.3f}')  # 1.0

# Verify genuine tripartite entanglement
xxx = result[0].data.evs
zz_sum = result[1].data.evs + result[2].data.evs + result[3].data.evs

if abs(xxx + 1.0) < 0.01 and abs(zz_sum - 3.0) < 0.01:
    print('\n✓ GHZ state verified!')
    print('✓ All three qubits are genuinely entangled')
else:
    print('\n✗ Not a valid GHZ state')

print('\n💡 EXAM TIP: GHZ verification uses multiple observables')
print('💡 Perfect correlations = perfect entanglement')

## Part 8: Runtime Estimator with Error Mitigation

### Runtime Configuration

**EXAM CRITICAL**: Know how to configure Runtime Estimator!

In [ ]:
# Runtime Estimator configuration (pseudocode for exam)
print('Runtime Estimator Pattern:')
print("""\nfrom qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Options

# Setup service
service = QiskitRuntimeService(channel='ibm_quantum')
backend = service.backend('ibm_brisbane')

# Configure options
options = Options()
options.resilience_level = 1  # M3 error mitigation
options.optimization_level = 3  # Aggressive optimization
options.execution.shots = 4096  # Number of shots

# Create Runtime Estimator
estimator = Estimator(backend=backend, options=options)

# Run (same API as StatevectorEstimator!)
job = estimator.run([(qc, observable)])
result = job.result()
expectation = result[0].data.evs
""")

print('\n⚠️ EXAM CRITICAL: resilience_level Options')
print('  0 = No error mitigation (fast but noisy)')
print('  1 = M3 mitigation (balanced)')
print('  2 = ZNE + PEC (slow but accurate)')

print('\n✓ Runtime Estimator API identical to StatevectorEstimator')
print('✓ Options object controls error mitigation')
print('✓ resilience_level=1 is recommended default')